In [1]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="phi3")

In [2]:
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType

In [3]:
# Connect to Milvus
connections.connect(
    host="localhost", # Replace with your Milvus server IP
    port="19530"
)

In [23]:
# Create schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="judgementChunkId", dtype=DataType.INT64),
    FieldSchema(name="judgementVector", dtype=DataType.FLOAT_VECTOR, dim=3072), # Vector field for film vectors
    FieldSchema(name="judgementText", dtype=DataType.VARCHAR, max_length=20000),
    FieldSchema(name="judgementFileName", dtype=DataType.VARCHAR, max_length=200)]

In [24]:
schema = CollectionSchema(fields=fields,enable_dynamic_field=False)

In [53]:
# collection_name = "judgement_collection"

# # Load the collection
# collection = Collection(name=collection_name)

# # Drop (delete) the collection
# collection.drop()

In [25]:
# Create collection
collection = Collection(name="judgement_collection", schema=schema)

In [26]:
# Create index for each vector field
index_params = {
    "metric_type": "L2",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 128},
}

In [28]:
collection.create_index("judgementVector", index_params)

Status(code=0, message=)

In [31]:
import pandas as pd
df = pd.read_csv('/home/ubuntu/ram/code/data/cleaned_combined_output.csv')

In [39]:
def word_count(text):
    return len(str(text).split())


In [40]:
df['judgment'].apply(word_count).max()

376698

In [41]:
df['judgment'].apply(word_count).mean()

1830.5832429213162

In [32]:
df_simple = df.head(10)

In [37]:
df_simple.columns

Index(['meta_data', 'judgment', 'file_name'], dtype='object')

In [42]:
from joblib import Parallel, delayed
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [44]:

# # Define the function that will process each row
# def process_row(idx, row):
#     splitter = RecursiveCharacterTextSplitter(
#     chunk_size=100,  # Maximum size of each chunk
#     chunk_overlap=20  # Overlap between chunks to maintain context
# )
#     # Split the text data
#     splits = splitter.split_text(str(row['judgment']))
#     results = []
#     for i,text in enumerate(splits):
#         entity = {"judgementChunkId":i,
#                   "judgementVector":embeddings.embed_query(text),
#                   "judgementText":text,
#                   "judgementFileName":row['file_name']}
#         results.append(entity)
#     chunk_size = 10
#     for i in range(0, len(results), chunk_size):
#         collection.insert(results[i:i + chunk_size])

# # Use joblib's Parallel and delayed to parallelize the loop
# Parallel(n_jobs=-1)(delayed(process_row)(idx, row) for idx, row in df_simple.iterrows())

PicklingError: Could not pickle the task to send it to the workers.

In [45]:
from joblib import Parallel, delayed
import pandas as pd

# Define the objects outside the function
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,  # Maximum size of each chunk
    chunk_overlap=20  # Overlap between chunks to maintain context
)

# Define the function that will process each row
def process_row(idx, row):
    splits = splitter.split_text(str(row['judgment']))
    results = []
    for i, text in enumerate(splits):
        entity = {
            "judgementChunkId": i,
            "judgementVector": embeddings.embed_query(text),
            "judgementText": text,
            "judgementFileName": row['file_name']
        }
        results.append(entity)
    
    chunk_size = 10
    for i in range(0, len(results), chunk_size):
        collection.insert(results[i:i + chunk_size])

# Use joblib's Parallel and delayed to parallelize the loop
# Use threading backend to avoid pickling issues
Parallel(n_jobs=-1, backend="threading")(delayed(process_row)(idx, row) for idx, row in df_simple.iterrows())


[None, None, None, None, None, None, None, None, None, None]

In [1]:
# Print all the records
from pymilvus import Collection

# Load the collection by its name
collection_name = "judgement_collection"  # Replace with your collection name
collection = Collection(name=collection_name)
collection.load()
query_expr = ""  # An empty string will retrieve all documents
# Specify the fields you want to retrieve
output_fields = ["id","judgementChunkId","judgementText", "judgementFileName"]  # Add all fields you want to prin
# Perform the query
results = collection.query(expr=query_expr, output_fields=output_fields,limit=1000)



ConnectionNotExistException: <ConnectionNotExistException: (code=1, message=should create connection first.)>

In [12]:
from pymilvus import Collection

# Load the collection by its name
collection_name = "judgement_collection"  # Replace with your collection name
collection = Collection(name=collection_name)
collection.load()

# Get the total number of entities in the collection
total_records = collection.num_entities

# Print the total number of records
print(f"Total number of records in the collection: {total_records}")

Total number of records in the collection: 115608
